<a href="https://colab.research.google.com/github/lefitzpatrick/BIL_SA_workflow/blob/main/download_nwm_multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install needed libraries
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00


In [2]:
# Import libraries
import os
import urllib.request
import netCDF4
from netCDF4 import Dataset
from datetime import datetime
import pandas as pd
import numpy as np

In [11]:
# Mount my google drive
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/MyDrive/BIL SA Project/Modeling/Inventories/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Presets
# Location of the NWM forecasts
base_url = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/v3.0/'
# Grab today's date
YYYYMMDD = datetime.today().strftime('%Y%m%d')
print(YYYYMMDD)
#forecast = "long_range_mem"
members = 1,2,3,4
hour = np.arange(6,721,6)
print(hour)
lakes = ['Superior','Erie','Ontario','MichHuron']
# Name output file
outfile = "total_nwm_runoff_cms_"+YYYYMMDD+".csv"
# Path you want it to save the output file to
path = '/content/drive/MyDrive/'

20240417
[  6  12  18  24  30  36  42  48  54  60  66  72  78  84  90  96 102 108
 114 120 126 132 138 144 150 156 162 168 174 180 186 192 198 204 210 216
 222 228 234 240 246 252 258 264 270 276 282 288 294 300 306 312 318 324
 330 336 342 348 354 360 366 372 378 384 390 396 402 408 414 420 426 432
 438 444 450 456 462 468 474 480 486 492 498 504 510 516 522 528 534 540
 546 552 558 564 570 576 582 588 594 600 606 612 618 624 630 636 642 648
 654 660 666 672 678 684 690 696 702 708 714 720]


There are 4 ensemble members for each forecast. I have it loop through all 4 but it would be good to have a way for the user to be able to choose 1 member or all of them.

In [5]:
# Download NWM forecast file
#for i in ens:
for ens in range(1,4+1):
  for hr in range(6,721,6):
    filename = "nwm.t00z.long_range.channel_rt_"+str(ens)+".f"+str(hr).zfill(3)+".conus.nc"
    path = base_url+"nwm."+YYYYMMDD+"/long_range_mem"+str(ens)+"/"+filename
    print(filename)
    urllib.request.urlretrieve(path,filename)


nwm.t00z.long_range.channel_rt_1.f006.conus.nc
nwm.t00z.long_range.channel_rt_1.f012.conus.nc
nwm.t00z.long_range.channel_rt_1.f018.conus.nc
nwm.t00z.long_range.channel_rt_1.f024.conus.nc
nwm.t00z.long_range.channel_rt_1.f030.conus.nc
nwm.t00z.long_range.channel_rt_1.f036.conus.nc
nwm.t00z.long_range.channel_rt_1.f042.conus.nc
nwm.t00z.long_range.channel_rt_1.f048.conus.nc
nwm.t00z.long_range.channel_rt_1.f054.conus.nc
nwm.t00z.long_range.channel_rt_1.f060.conus.nc
nwm.t00z.long_range.channel_rt_1.f066.conus.nc
nwm.t00z.long_range.channel_rt_1.f072.conus.nc
nwm.t00z.long_range.channel_rt_1.f078.conus.nc
nwm.t00z.long_range.channel_rt_1.f084.conus.nc
nwm.t00z.long_range.channel_rt_1.f090.conus.nc
nwm.t00z.long_range.channel_rt_1.f096.conus.nc
nwm.t00z.long_range.channel_rt_1.f102.conus.nc
nwm.t00z.long_range.channel_rt_1.f108.conus.nc
nwm.t00z.long_range.channel_rt_1.f114.conus.nc
nwm.t00z.long_range.channel_rt_1.f120.conus.nc
nwm.t00z.long_range.channel_rt_1.f126.conus.nc
nwm.t00z.long

This section reads in each of Yi's csv files that contain all of the feature_ids for the streams that discharge into each lake.

In [6]:
# Lake Superior
su_links = pd.read_csv(dir+'NWM_runoff/SU_link.csv')
id_SU = su_links['ID']
# Lake Erie
er_links = pd.read_csv(dir+'NWM_runoff/ER_link.csv')
id_ER = er_links['ID']
# Lake Ontario
on_links = pd.read_csv(dir+'NWM_runoff/ON_link.csv')
id_ON = on_links['ID']
# Lake Erie
mh_links = pd.read_csv(dir+'NWM_runoff/MIHU_link.csv')
id_MH = mh_links['ID']

Now we need to loop through each of the downloaded files; opening them, extracting the streamflows and adding them up into each lake for the total runoff. This is not the most efficient way, however, by creating a new dataframe with the streamflow from each of the streams, we have the ability to go back and look at them individually if anything strange appears.

In [7]:
# Set up empty data frames where the feature ids, pulled from the csvs above, are
# the indices and there's an empty column to insert the streamflow for each file.
df_su_hr = pd.DataFrame(index=id_SU,columns=['streamflow'])
df_er_hr = pd.DataFrame(index=id_ER,columns=['streamflow'])
df_on_hr = pd.DataFrame(index=id_ON,columns=['streamflow'])
df_mh_hr = pd.DataFrame(index=id_MH,columns=['streamflow'])

# Empty data frame to insert the total runoff for each hour.
df_su = pd.DataFrame(index=hour,columns=['streamflow'])
df_er = pd.DataFrame(index=hour,columns=['streamflow'])
df_on = pd.DataFrame(index=hour,columns=['streamflow'])
df_mh = pd.DataFrame(index=hour,columns=['streamflow'])

# Save each ensemble total streamflows for each lake.
df_lakes = pd.DataFrame(index=members,columns=lakes)

In [8]:
for ens in range(1,5):
  #for hr in hour:
  for hr in range(6,721,6):
    filename = "nwm.t00z.long_range.channel_rt_"+str(ens)+".f"+str(hr).zfill(3)+".conus.nc"
    file = Dataset(filename, mode='r', format="NETCDF4")
    feature_id = file.variables['feature_id']
    streamflow = file.variables['streamflow'] #streamflow is in m3/s

    for f_su in id_SU:
      df_su_hr['streamflow'][f_su]= streamflow[feature_id == f_su]
    for f_er in id_ER:
      df_er_hr['streamflow'][f_er]= streamflow[feature_id == f_er]
    for f_on in id_ON:
      df_on_hr['streamflow'][f_on]= streamflow[feature_id == f_on]
    for f_mh in id_MH:
      df_mh_hr['streamflow'][f_mh]= streamflow[feature_id == f_mh]
    file.close()

    df_su['streamflow'][hr] = df_su_hr['streamflow'].sum()
    df_er['streamflow'][hr] = df_er_hr['streamflow'].sum()
    df_on['streamflow'][hr] = df_on_hr['streamflow'].sum()
    df_mh['streamflow'][hr] = df_mh_hr['streamflow'].sum()
    print(ens,hr,df_su['streamflow'][hr])

  df_lakes[lakes[0]][ens] = df_su['streamflow'].sum()
  df_lakes[lakes[1]][ens] = df_er['streamflow'].sum()
  df_lakes[lakes[2]][ens] = df_on['streamflow'].sum()
  df_lakes[lakes[3]][ens] = df_mh['streamflow'].sum()
  print(ens,df_lakes[lakes[0]][ens])

1 6 1757.6999607123435
1 12 1720.4999615438282
1 18 1673.9999625831842
1 24 1627.4999636225402
1 30 1590.299964454025
1 36 1553.0999652855098
1 42 1515.8999661169946
1 48 1487.9999667406082
1 54 1450.799967572093
1 60 1422.8999681957066
1 66 1394.9999688193202
1 72 1367.0999694429338
1 78 1329.8999702744186
1 84 1301.9999708980322
1 90 1283.3999713137746
1 96 1255.4999719373882
1 102 1236.8999723531306
1 108 1218.299972768873
1 114 1199.6999731846154
1 120 1171.799973808229
1 126 1153.1999742239714
1 132 1134.5999746397138
1 138 1115.9999750554562
1 144 1097.3999754711986
1 150 1078.799975886941
1 156 1060.1999763026834
1 162 1041.5999767184258
1 168 1022.9999771341681
1 174 1004.3999775499105
1 180 985.7999779656529
1 186 967.1999783813953
1 192 948.5999787971377
1 198 939.2999790050089
1 204 920.6999794207513
1 210 902.0999798364937
1 216 892.7999800443649
1 222 874.1999804601073
1 228 864.8999806679785
1 234 846.2999810837209
1 240 836.9999812915921
1 246 818.3999817073345
1 252 799

In [9]:
print(df_lakes)

       Superior          Erie       Ontario      MichHuron
1  85160.098097  32324.209277  34796.599222  111990.107497
2  85504.198089  32454.819275  34937.199219  112442.617487
3  85113.598098  32306.559278  34777.599223  111928.957498
4  87987.298033  33397.329254  35951.799196  115708.027414


In [16]:
# Saves a csv file with total flow into each lake [cms] for that time period
# This is for one time period
df_lakes.to_csv(dir+outfile, sep='\t')